# Avaliadores de Resumo

### Configuração

In [ ]:
# Você pode defini-las diretamente
import os
os.environ["GOOGLE_API_KEY"] = ""
os.environ["LANGSMITH_API_KEY"] = ""
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "langsmith-academy"

In [ ]:
# Ou você pode usar um arquivo .env
from dotenv import load_dotenv
load_dotenv(dotenv_path="../../.env", override=True)

### Tarefa

Nossa tarefa aqui é analisar a toxicidade de declarações aleatórias, classificando-as como `Tóxico` ou `Não tóxico`.

Dê uma olhada no nosso dataset!

In [ ]:
from langsmith import Client

client = Client()
dataset = client.clone_public_dataset(
    "https://smith.langchain.com/public/89ef0d44-a252-4011-8bb8-6a114afc1522/d"
)

Este é um classificador de toxicidade simples!

In [ ]:
import sys
sys.path.append('../../')
from gemini_utils import call_gemini_chat
from pydantic import BaseModel, Field

class Toxicity(BaseModel):
    toxicity: str = Field(description="""'Tóxico' se a declaração for tóxica, 'Não tóxico' se a declaração não for tóxica.""")

def good_classifier(inputs: dict) -> dict:
    messages = [
        {
            "role": "user",
            "content": f"Esta é a declaração: {inputs['statement']}. Classifique como 'Tóxico' ou 'Não tóxico'."
        }
    ]
    
    response = call_gemini_chat("gemini-1.5-flash", messages, 0.0)
    
    # Extrair classificação da resposta (simplificado)
    if "tóxico" in response.lower() and "não" not in response.lower():
        toxicity_score = "Tóxico"
    else:
        toxicity_score = "Não tóxico"
    
    return {"class": toxicity_score}

### Avaliador de Resumo

Estes são os campos aos quais as funções de avaliador de resumo têm acesso:
- `inputs: list[dict]`: Uma lista de entradas dos exemplos em nosso dataset
- `outputs: list[dict]`: Uma lista das saídas dict produzidas ao executar nosso alvo sobre cada entrada
- `reference_outputs: list[dict]`: Uma lista de reference_outputs dos exemplos em nosso dataset
- `runs: list[Run]`: Uma lista dos objetos Run de executar nosso alvo sobre o dataset.
- `examples: list[Example]`: Uma lista dos Examples completos do dataset, incluindo as entradas do exemplo, saídas (se disponível) e metadata (se disponível).

Agora vamos definir nosso avaliador de resumo! Aqui, vamos calcular o f1-score, que é uma combinação de precisão e recall.

Este tipo de métrica só pode ser calculada sobre todos os exemplos em nosso experimento, então nosso avaliador recebe uma lista de saídas e uma lista de reference_outputs.

In [ ]:
def f1_score_summary_evaluator(outputs: list[dict], reference_outputs: list[dict]) -> dict:
    true_positives = 0
    false_positives = 0
    false_negatives = 0
    for output_dict, reference_output_dict in zip(outputs, reference_outputs):
        output = output_dict["class"]
        reference_output = reference_output_dict["class"]
        if output == "Tóxico" and reference_output == "Tóxico":
            true_positives += 1
        elif output == "Tóxico" and reference_output == "Não tóxico":
            false_positives += 1
        elif output == "Não tóxico" and reference_output == "Tóxico":
            false_negatives += 1

    if true_positives == 0:
        return {"key": "f1_score", "score": 0.0}

    precision = true_positives / (true_positives + false_positives)
    recall = true_positives / (true_positives + false_negatives)
    f1_score = 2 * (precision * recall) / (precision + recall)
    return {"key": "f1_score", "score": f1_score}

Note que passamos `f1_score_summary_evaluator` como um avaliador de resumo!

In [ ]:
results = client.evaluate(
    good_classifier,
    data=dataset,
    summary_evaluators=[f1_score_summary_evaluator],
    experiment_prefix="Bom classificador"
)